In [2]:
import ts_y_wrapper as ts
import glob
import array as ar
import numpy as np

In [15]:
max_cluster_size = 450
cluster_hist = np.zeros(max_cluster_size)

In [5]:
file_regex = ("/opt/workspace/msc_project/simulations"
                      "/QA_tests/cluster_version/feature_tests"
                      "/ts_measure_works/new_tspoststats"
                      "/timestep_000[0-1][0-9][0-9].vtu")

file_regex_2 = ("/mnt/c/Users/yoavr/Desktop/paraview_pipeline"
                "/hello_chemfarm/pert_pan_sim_6/timesteps"
                "/timestep_f[0-9]w[0-9]_000[0-1][0-9][0-9].vtu")

files = glob.glob(file_regex_2)
files[:6]

['/mnt/c/Users/yoavr/Desktop/paraview_pipeline/hello_chemfarm/pert_pan_sim_6/timesteps/timestep_f0w0_000000.vtu',
 '/mnt/c/Users/yoavr/Desktop/paraview_pipeline/hello_chemfarm/pert_pan_sim_6/timesteps/timestep_f0w0_000001.vtu',
 '/mnt/c/Users/yoavr/Desktop/paraview_pipeline/hello_chemfarm/pert_pan_sim_6/timesteps/timestep_f0w0_000002.vtu',
 '/mnt/c/Users/yoavr/Desktop/paraview_pipeline/hello_chemfarm/pert_pan_sim_6/timesteps/timestep_f0w0_000003.vtu',
 '/mnt/c/Users/yoavr/Desktop/paraview_pipeline/hello_chemfarm/pert_pan_sim_6/timesteps/timestep_f0w0_000004.vtu',
 '/mnt/c/Users/yoavr/Desktop/paraview_pipeline/hello_chemfarm/pert_pan_sim_6/timesteps/timestep_f0w0_000005.vtu']

In [7]:
for file in files[:6]:
    vesicle=ts.parseDump(file)
    cstlist=ts.init_cluster_list()
    ts.clusterize_vesicle(vesicle, cstlist)    
    for j in range(cstlist.contents.n):
        size=cstlist.contents.cluster[j].contents.nvtx
        cluster_hist[size-1]+=1.0000

In [23]:
vesicle.contents.tape.contents.vicsek_model

4621819117588971520

In [ ]:
#for i in range(n):
#    f=open('cluster_dist_passive_E_2_pt_5_c_350.dat','a')
#    f.write('{}, {} \n'.format(i+1,cl_array[i]/icount))